# 05 - Topic Modeling - Exercise 3

In [1]:
import pandas as pd
import numpy as np
import gensim

C:\Users\Inês Valentim\Anaconda3\lib\site-packages\gensim\utils.py:840: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
C:\Users\Inês Valentim\Anaconda3\lib\site-packages\gensim\utils.py:1015: UserWarning: Pattern library is not installed, lemmatization won't be available.
  warnings.warn("Pattern library is not installed, lemmatization won't be available.")


In [2]:
### ONLY RUN ONCE ####
# import nltk
# nltk.download('wordnet')

In [3]:
from nltk.tokenize import regexp_tokenize
from nltk.corpus import stopwords
# from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

# create DataFrame
path_emails = 'hillary-clinton-emails/Emails.csv'
emails = pd.read_csv(path_emails, usecols = ['SenderPersonId', 'ExtractedSubject', 'ExtractedBodyText'])

# drop records where both ExtractedSubject and ExtractedBodyText are NaN
emails.dropna(axis=0, thresh=2, subset=[['ExtractedSubject', 'ExtractedBodyText']], inplace=True)
emails.fillna(' ', inplace = True)

# group emails by sender
emails_raw = []

for sender in emails.groupby('SenderPersonId'):
    grouped_emails = ''
    
    for subject, body in zip(sender[1]['ExtractedSubject'], sender[1]['ExtractedBodyText']):
        grouped_emails += (subject + ' ' + body + ' ')
    
    emails_raw.append(grouped_emails)

# tokenization
email_tokens = []

for email in emails_raw:
    email_tokens.append(regexp_tokenize(email, pattern='\w+'))

# remove stopwords and digits/numbers
stop_words = set(stopwords.words('english'))
stopwords_emails = ['fyi', 'fm', 'am', 'pm', 'n\'t', 'sent', 'from', 'to', 'subject', 'fw', 'fwd', 'fvv',
                    'cc', 'bcc', 'attachments', 're', 'date', 'html', 'php']
stop_words.update(stopwords_emails)

email_clean_tokens = []

for email in email_tokens:
    clean_tokens = [token for token in email if token.lower() not in stop_words and token.isdigit() == False]
    email_clean_tokens.append(clean_tokens)

# lemmatization
# stemmer = PorterStemmer()
wnl = WordNetLemmatizer()

email_lemma = []

for email in email_clean_tokens:
    lemma = [wnl.lemmatize(token.lower()) for token in email]
    email_lemma.append(lemma)
    
# remove tokens that are too small
email_clean = []

for email in email_lemma:
    clean = [token for token in email if len(token) > 1]
    email_clean.append(clean)

In [4]:
dictionary = gensim.corpora.Dictionary(email_clean)
corpus = [dictionary.doc2bow(email) for email in email_clean]

In [5]:
number_topics = [5, 10, 15, 20, 25, 30, 35, 40, 45, 50]

for n in number_topics:
    lda_model = gensim.models.ldamodel.LdaModel(corpus, num_topics=n, id2word=dictionary, passes=10)
   
    print('#### Model with', n, 'topics ####')
   
    topic_id = 1
   
    for topic in lda_model.show_topics(num_topics=n, num_words=10, log=False, formatted=False):
        
        string_words = ''
        
        for word in topic[1]:
            string_words += (word[0] + ' ')
        
        print('topic #', topic_id, ':', string_words)

        topic_id += 1

#### Model with 5 topics ####
topic # 1 : state call would department also house government new one said 
topic # 2 : state clinton said haiti one say call would see work 
topic # 3 : obama would party american president state israel one new republican 
topic # 4 : call pls ok get know see thx want tomorrow talk 
topic # 5 : office secretary state meeting room department arrive en route depart 
#### Model with 10 topics ####
topic # 1 : jeffrey feltman office cheryl mill state secretary call joanne behalf 
topic # 2 : state clinton said secretary would time one new president war 
topic # 3 : state secretary office call meeting department room time house en 
topic # 4 : state haiti cdm call clinton work see would secretary need 
topic # 5 : call state pls see thx get ok know tomorrow talk 
topic # 6 : group stonebridge albright contact new state merger wendy information work 
topic # 7 : state clinton would treaty start new one senate call president 
topic # 8 : palau state united uighu